# Zero Curve Construction and Analysis

## Overview
In this notebook we turn our attention from the design and creation of financial instrument objects  (eg bonds and bills) to the construction of a simple zero curve.  

A zero curve is a collection of dates and discount factors which collectively describe the term structure of interest rates through time and permit us to discount future cashflows to today.

Having a zero curve enable us to perform valuation and other calcualations in relation to financial instruments and cashflows. 

## Key Concepts
- **Zero Rates**: Interest rates for zero-coupon bonds of different maturities
- **Discount Factors**: Present values of 1 unit of currency paid at future dates
- **Exponential Interpolation**: A method for estimating rates between known points on the curve
- **Amount at Maturity (AtMat)**: Future value based on zero rates

## Dependencies
The notebook uses standard Python libraries for mathematical operations and data manipulation:
- **math**: For basic mathematical operations (exponential, logarithm functions)
- **numpy**: For numerical computations and array operations
- **pandas**: For data manipulation and analysis with DataFrames
- **importlib**: For reloading modules during development and testing
- **tabulate**: For creating formatted tables from data structures

In [9]:
import math
import numpy as np
import importlib
import pandas as pd
import tabulate as tb

# The ZeroCurve Class

Before we can work with zero curves, we need to define the `ZeroCurve` class. This class is a simple collection of dates and discount factors which provides an ability to do NPV calculations as well as report rates and factors implied by the curve.

## Class Structure

The `ZeroCurve` class maintains four parallel lists to represent the yield curve:
- **maturities**: Time points (in years) where we have rate information
- **zero_rates**: Continuously compounded zero rates at each maturity
- **AtMats**: Amount at maturity values (future values of $1 invested at time 0)
- **discount_factors**: Present values of $1 received at each maturity

## Key Methods

1. **add_zero_rate(maturity, zero_rate)**: Add a point to the curve using a zero rate
2. **add_discount_factor(maturity, discount_factor)**: Add a point to the curve using a discount factor
3. **get_zero_rate(maturity)**: Retrieve the zero rate for any maturity (with interpolation)
4. **get_discount_factor(maturity)**: Retrieve the discount factor for any maturity (with interpolation)
5. **get_AtMat(maturity)**: Retrieve the amount at maturity for any time point
6. **npv(cash_flows)**: Calculate the net present value of a cash flow stream

## Exponential Interpolation Helper Function

The class uses a "helper" or "utility" function (the `exp_interp` function) to do exponential interpolation in order to estimate values for maturities that don't exactly match stored points. This ensures that rates interpolate smoothly and maintain the no-arbitrage property of continuously compounded rates.

In [10]:
class ZeroCurve:
    def __init__(self):
        # set up empty lists to store the curve data
        self.maturities = []
        self.zero_rates = []
        self.AtMats = []
        self.discount_factors = []
    
    def add_zero_rate(self, maturity, zero_rate):
        """Add a zero rate to the curve and calculate corresponding discount factor and AtMat"""
        self.maturities.append(maturity)
        self.zero_rates.append(zero_rate)
        self.AtMats.append(math.exp(zero_rate*maturity))
        self.discount_factors.append(1/self.AtMats[-1])

    def add_discount_factor(self, maturity, discount_factor):
        """Add a discount factor to the curve and calculate corresponding zero rate and AtMat"""
        self.maturities.append(maturity)
        self.discount_factors.append(discount_factor)
        self.AtMats.append(1/discount_factor)
        self.zero_rates.append(math.log(1/discount_factor)/maturity)
    
    def get_AtMat(self, maturity):
        """Get the amount at maturity for a given time point (with interpolation if needed)"""
        if maturity in self.maturities:
            return self.AtMats[self.maturities.index(maturity)]
        else:
            return exp_interp(self.maturities, self.AtMats, maturity)

    def get_discount_factor(self, maturity):
        """Get the discount factor for a given maturity (with interpolation if needed)"""
        if maturity in self.maturities:
            return self.discount_factors[self.maturities.index(maturity)]
        else:
            return exp_interp(self.maturities, self.discount_factors, maturity)

    def get_zero_rate(self, maturity):
        """Get the zero rate for a given maturity (with interpolation if needed)"""
        if maturity in self.maturities:
            return self.zero_rates[self.maturities.index(maturity)]
        else:
            return math.log(self.get_AtMat(maturity))/maturity
        
    def get_zero_curve(self):
        """Return the complete zero curve as maturities and discount factors"""
        return self.maturities, self.discount_factors
    
    def npv(self, cash_flows):
        """Calculate the net present value of a cash flow stream"""
        npv = 0
        for maturity in cash_flows.get_maturities():
            npv += cash_flows.get_cash_flow(maturity)*self.get_discount_factor(maturity)
        return npv
        

def exp_interp(xs, ys, x):
    """
    Interpolates a single point for a given value of x 
    using continuously compounded rates.

    Parameters:
    xs (list or np.array): Vector of x values sorted by x.
    ys (list or np.array): Vector of y values.
    x (float): The x value to interpolate.

    Returns:
    float: Interpolated y value.
    """
    xs = np.array(xs)
    ys = np.array(ys)
    
    # Find the interval [x0, x1] where x0 <= x <= x1
    idx = np.searchsorted(xs, x) - 1
    x0, x1 = xs[idx], xs[idx + 1]
    y0, y1 = ys[idx], ys[idx + 1]
    
    # Calculate the continuously compounded rate
    rate = (np.log(y1) - np.log(y0)) / (x1 - x0)
    
    # Interpolate the y value for the given x
    y = y0 * np.exp(rate * (x - x0))
    
    return y

---
**OOP Concept: Encapsulation**

Notice how the `ZeroCurve` class demonstrates **encapsulation** - one of the core principles of object-oriented programming we discussed earlier:

- The internal data structures (`self.maturities`, `self.zero_rates`, `self.AtMats`, `self.discount_factors`) are stored as attributes within the class
- Users don't need to know how the data is stored internally - they interact with the curve through well-defined methods like `add_zero_rate()` and `get_discount_factor()`
- The class automatically maintains consistency between different representations (zero rates, discount factors, amounts at maturity)
- This encapsulation hides the complexity of the calculations and data management from the user

This is exactly the principle we explored in the introductory OOP notebook - bundling data with the methods that operate on that data, and controlling access through a clean interface.

# Working with the ZeroCurve Class

The following section demonstrates the main functionality of the ZeroCurve class. We'll explore:

1. Creating and populating a zero curve
2. Calculating various measures (zero rates, discount factors, amounts at maturity)
3. Interpolating values for non-standard maturities
4. Presenting the curve data in different formats

The ZeroCurve class manages internal consistency between different representations of the same curve (zero rates, discount factors, and amounts at maturity) and provides interpolation capabilities for missing points.

We start by importing the ZeroCurve Class from the curve_classes_and_funcitons python file in the same way that we would import from any 3rd party library.  You should inspect the code in the class statement relating to the Zero Curve Class in this library.

In [11]:
# Create an instance of the ZeroCurve class
# This initializes an empty curve ready to accept rate data
zc = ZeroCurve()

# Add zero rates to the curve
# These rates represent annual interest rates for zero-coupon bonds
zc.add_zero_rate(0.5, 0.0125)  # 6-month rate
zc.add_zero_rate(1, 0.015)  # 1.5% for 1 year
zc.add_zero_rate(2, 0.025)  # 2.5% for 2 years
zc.add_zero_rate(3, 0.035)  # 3.5% for 3 years
zc.add_zero_rate(4, 0.045)  # 4.5% for 4 years

# Demonstrate retrieving zero rates
print("2.5-year zero rate:", zc.get_zero_rate(2.5))

# Calculate and display discount factors
# Discount factors represent the present value of 1 unit of currency
print("1-year discount factor:", zc.get_discount_factor(1))
print("2-year discount factor:", zc.get_discount_factor(2))
print("3-year discount factor:", zc.get_discount_factor(3))
print("4-year discount factor:", zc.get_discount_factor(4))

# Demonstrate interpolation for a non-standard maturity
maturity_lookup = 1.5
print(f"Zero rate for {maturity_lookup} years:", zc.get_zero_rate(maturity_lookup))
print(f"Amount at Maturity for {maturity_lookup} years:", zc.get_AtMat(maturity_lookup))
print(f"Discount factor for {maturity_lookup} years:", zc.get_discount_factor(maturity_lookup))

# Get the complete zero curve data
print("Complete zero curve:", zc.get_zero_curve())

# Create a pandas DataFrame for better data visualization and analysis
zcT = np.transpose(zc.get_zero_curve())
zc_dataframe = pd.DataFrame(zcT, columns=['Maturity', 'Discount Factor'])
zc_dataframe.set_index('Maturity', inplace=True)
print("\nZero Curve DataFrame:")
print(zc_dataframe)
zc_dataframe

2.5-year zero rate: 0.03100000000000001
1-year discount factor: 0.9851119396030628
2-year discount factor: 0.9512294245007139
3-year discount factor: 0.9003245225862656
4-year discount factor: 0.835270211411272
Zero rate for 1.5 years: 0.02166666666666669
Amount at Maturity for 1.5 years: 1.0330338931439726
Discount factor for 1.5 years: 0.968022449831306
Complete zero curve: ([0.5, 1, 2, 3, 4], [0.9937694906233947, 0.9851119396030628, 0.9512294245007139, 0.9003245225862656, 0.835270211411272])

Zero Curve DataFrame:
          Discount Factor
Maturity                 
0.5              0.993769
1.0              0.985112
2.0              0.951229
3.0              0.900325
4.0              0.835270


,Discount Factor
Maturity,
0.5,0.993769
1.0,0.985112
2.0,0.951229
3.0,0.900325
4.0,0.835270


---
**OOP Concept: Abstraction**

The code above beautifully illustrates **abstraction** - another fundamental OOP principle we discussed earlier.

Notice how we can use the `ZeroCurve` class without understanding its internal implementation:
- When we call `zc.get_zero_rate(2.5)`, we get the interpolated rate without needing to know about the exponential interpolation algorithm
- When we call `zc.get_discount_factor(1.5)`, the class handles all the mathematics behind the scenes
- We don't need to worry about how the class maintains consistency between zero rates, discount factors, and amounts at maturity

**Abstraction hides complexity** and exposes only what's necessary through a simple, intuitive interface. We can work at a higher level of thinking - focusing on *what* we want (a discount factor, a zero rate) rather than *how* it's calculated.

This is like driving a car - you use the steering wheel and pedals without needing to understand the engine mechanics. Similarly, we use `get_discount_factor()` without needing to understand exponential interpolation or logarithmic transformations.

This abstraction makes our code more usable, reduces cognitive load, and allows us to build complex financial models without getting lost in implementation details.

# Using the Zero Curve for Instrument Valuation

Now that we have created our zero curve `zc`, we can use it to value various financial instruments including:
- Simple cash flows
- Bank bills
- Bonds
- Portfolios of instruments

We'll continue using the zero curve object created earlier in this notebook.

### Import instrument classes

In the previous notebook, we created a series of financial instrument classes.   To avoid rewriting the class statements associated with this instruments every time we want to use them, we can instead store them in a module for use (by importing in the same way that we do for 3rd party libraries) in different contexts and applications.   Take a look at the `instrument_classes.py` file which now serves as this library.  

Below, we import the instrument classes (CashFlows, Bank_bill, Bond, Portfolio) from this module to create the financial instruments.

Then we will be able to value these instruments using the zero curve we created above.

In [12]:
import instrument_classes as instruments
importlib.reload(instruments)

<module 'instrument_classes' from '/workspaces/FM_yield_curve/instrument_classes.py'>

### Set up a simple cash_flows object containing to cashflows manually added

Notice that we are using the classes contained within `instruments` which is now shorthand for our instrument_classes module.

In [13]:
# create an instance of the CashFlows class called my_cash_flows
# remember that an instance is a specific object created from a class blueprint/template
my_cash_flows = instruments.CashFlows()

# add a cash flow of 1000 at time 1
my_cash_flows.add_cash_flow(1, 1000)

# add a cash flow of 1500 at time 2.5
my_cash_flows.add_cash_flow(2.5, 1500)

print(my_cash_flows.get_cash_flows())
print(my_cash_flows.get_maturities())
print(my_cash_flows.get_amounts())
print(zc.npv(my_cash_flows))

# create a new dataframe called cash_flows_df to store the cash flows
cash_flows_df = pd.DataFrame(data={'maturity': my_cash_flows.get_maturities(), 'amount': my_cash_flows.get_amounts()})
# set the index of the dataframe to be the maturity
cash_flows_df.set_index('maturity', inplace=True)
cash_flows_df

[(1, 1000), (2.5, 1500)]
[1, 2.5]
[1000, 1500]
2373.252476198018


,amount
maturity,
1.0,1000
2.5,1500


### Set up and value a bank_bill

In [14]:
# create an instance of the bank_bill class called my_bank_bill
my_bank_bill = instruments.Bank_bill()
my_bank_bill.set_ytm(0.06)
my_bank_bill.set_cash_flows()

bank_bill_cashflows = my_bank_bill.get_cash_flows()
print(bank_bill_cashflows)
print(zc.npv(my_bank_bill))

# create a new dataframe called cash_flows_df to store the cash flows
bill_cash_flows_df = pd.DataFrame(data={'Maturity': my_bank_bill.get_maturities(), 'Amount': my_bank_bill.get_amounts()})
# set the index of the dataframe to be the maturity
bill_cash_flows_df.set_index('Maturity', inplace=True)
bill_cash_flows_df

[(0, -98.52216748768474), (0.25, 100)]
0.249012453953398


,Amount
Maturity,
0.00,-98.522167
0.25,100.000000


### Set up and value a bond

In [15]:
# create an instance of the bond class called my_bond
my_bond = instruments.Bond()
my_bond.set_maturity(1)
my_bond.set_coupon(0.05)
my_bond.set_frequency(2)
my_bond.set_face_value(100)
my_bond.set_ytm(0.05)

my_bond.set_cash_flows()
bond_cashflows = my_bond.get_cash_flows()
print("The cashflows of the bond are : " + str(bond_cashflows))
print("The PV of the bond is : " + str(zc.npv(my_bond)))
print("")

# create a new dataframe called cash_flows_df to store the cash flows
bond_cash_flows_df = pd.DataFrame(data={'Maturity': my_bond.get_maturities(), 'Amount': my_bond.get_amounts()})
# set the index of the dataframe to be the maturity
bond_cash_flows_df.set_index('Maturity', inplace=True)

print("Here are the cashflows of the bond in a dataframe : ")
print(bond_cash_flows_df.to_string())
print("")
print("Here are the cashflows of the bond in a table using the tabulate library: ")
print(tb.tabulate(bond_cash_flows_df, headers='keys', tablefmt='psql'))

test_price = (my_bond.coupon/my_bond.frequency*my_bond.face_value)*(1-(1+my_bond.ytm/my_bond.frequency)**(-my_bond.maturity*my_bond.frequency))/(my_bond.ytm/my_bond.frequency) \
          + my_bond.face_value/((1 + my_bond.ytm/my_bond.frequency)**(my_bond.maturity*my_bond.frequency))
print(test_price)

The cashflows of the bond are : [(0, -99.99999999999999), (0.5, 2.5), (1, 102.5)]
The PV of the bond is : 1.583902529149313

Here are the cashflows of the bond in a dataframe : 
          Amount
Maturity        
0.0       -100.0
0.5          2.5
1.0        102.5

Here are the cashflows of the bond in a table using the tabulate library: 
+------------+----------+
|   Maturity |   Amount |
|------------+----------|
|        0   |   -100   |
|        0.5 |      2.5 |
|        1   |    102.5 |
+------------+----------+
99.99999999999999


### Set up a portfolio containing some bank bills and bonds
Now we create a portfolio and add the bond and bill instantiated above, together with a a new one of each.  

In [16]:
# create an instance of the portfolio class called my_portfolio
my_portfolio = instruments.Portfolio()

# create another instance of the bond class called my_bond_2
my_bond_2 = instruments.Bond()
my_bond_2.set_maturity(2)
my_bond_2.set_coupon(0.08)
my_bond_2.set_frequency(4)
my_bond_2.set_face_value(1000)
my_bond_2.set_ytm(0.04)
my_bond_2.set_cash_flows()

# add the bonds to the portfolio
my_portfolio.add_bond(my_bond_2)
my_portfolio.add_bond(my_bond)

print("The bonds in the portfolio are : " + str(my_portfolio.get_bonds()))
bond_cashflows = my_bond_2.get_cash_flows()
print("The cashflows of my_bond_2 are : " + str(bond_cashflows))
print("The PV of my_bond_2 is : " + str(zc.npv(my_bond)))
print("")

# create an instance of the bank_bill class called bank_bill_2
bank_bill_2 = instruments.Bank_bill()
bank_bill_2.set_maturity(.5)
bank_bill_2.set_ytm(0.07)
bank_bill_2.set_cash_flows()

my_portfolio.add_bank_bill(bank_bill_2)
my_portfolio.add_bank_bill(my_bank_bill)

my_portfolio.set_cash_flows()
portfolio_cashflows = my_portfolio.get_cash_flows()
print("The cashflows of the portfolio are : ")

portfolio_cashflows_df = pd.DataFrame(data={'Maturity': my_portfolio.get_maturities(), 'Amount': my_portfolio.get_amounts()})
print(portfolio_cashflows_df.to_string())

print("The PV of the portfolio is : " + str(zc.npv(my_portfolio)))
print("")


The bonds in the portfolio are : [<instrument_classes.Bond object at 0x7799307b0fb0>, <instrument_classes.Bond object at 0x7799542c5af0>]
The cashflows of my_bond_2 are : [(0, -1076.5167775176878), (0.25, 20.0), (0.5, 20.0), (0.75, 20.0), (1.0, 20.0), (1.25, 20.0), (1.5, 20.0), (1.75, 20.0), (2, 1020.0)]
The PV of my_bond_2 is : 1.583902529149313

The cashflows of the portfolio are : 
    Maturity       Amount
0       0.00 -1076.516778
1       0.25    20.000000
2       0.50    20.000000
3       0.75    20.000000
4       1.00    20.000000
5       1.25    20.000000
6       1.50    20.000000
7       1.75    20.000000
8       2.00  1020.000000
9       0.00  -100.000000
10      0.50     2.500000
11      1.00   102.500000
12      0.00   -96.618357
13      0.50   100.000000
14      0.00   -98.522167
15      0.25   100.000000
The PV of the portfolio is : -3199.380819599886



---
**OOP Concept: Polymorphism in Action**

Throughout this valuation section, we've seen a powerful demonstration of **polymorphism** - one of the key principles of object-oriented programming discussed in the introductory OOP notebook.

Notice how the `zc.npv()` method works seamlessly with many different types of objects:
- Simple `CashFlows` objects
- `Bank_bill` objects
- `Bond` objects  
- `Portfolio` objects containing multiple instruments

The `npv()` method doesn't need to know what specific type of object it's receiving. It only needs to know that the object implements the required interface:
- A `get_maturities()` method that returns a list of time points
- A `get_cash_flow(maturity)` method that returns the cash flow amount at a given time

This is the essence of polymorphism: writing code once that can work with many different types of objects, as long as they share a common interface. The same method call (`zc.npv(object)`) produces correct results whether we're valuing a simple cash flow stream or a complex portfolio of bonds and bills.

**Benefits of Polymorphism:**
- **Flexibility**: New instrument types can be added without modifying the `ZeroCurve` class
- **Reusability**: The same valuation logic works for all current and future instrument types
- **Maintainability**: Changes to valuation logic only need to be made in one place
- **Extensibility**: Any new class that implements `get_maturities()` and `get_cash_flow()` can be valued automatically

This design principle makes our financial modeling framework both powerful and elegant!